In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Importing Required Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import norm, boxcox
from scipy import stats
from imblearn.over_sampling import SMOTE


**Loading the red wine quality dataset**

In [ ]:
df = pd.read_csv('../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')
df.head()

# **Exploring the dataset**

1. **Checking No. of rows and columns**

In [ ]:
print('Shape of the dataset')
print(df.shape)


2. **Checking for null values**

In [ ]:
print('Checking for the null values')
print(df.isnull().sum())

3. **Checking the data types of each feautes and nows of rows and null count**

In [ ]:
df.info()

4. **Cheking the no. of rows, mean value of each rows, std and five number summary**

In [ ]:
df.describe()

# EDA

**Quality count**

In [ ]:
sns.countplot(x ='quality', data = df)

In [ ]:
from collections import Counter
Counter(df['quality'])

**Alcohol VS Quality**

In [ ]:
sns.boxplot(x="quality",y="alcohol", data=df)

we can see that as we are increasing the alcholo the quality of wine is improving.

**Residual sugar VS Quality**

In [ ]:
plot = plt.figure(figsize=(15,7))
sns.boxplot(x="quality",y="residual sugar", data=df)

we can see that there is no correlation between the residual sugar and quality.
So, we will drop this column as it has not contribution in determining the qulaity.

**Instead of checking one by one we can use correlation matrix and heat map to analyse the correlation between each feature at once**

**volatile acidity VS Quality**

In [ ]:
sns.boxplot(x="quality",y="volatile acidity", data=df)

**PH VS Quality**

In [ ]:
plot = plt.figure(figsize=(15,7))
sns.boxplot(x="quality",y="pH", data=df)

In [ ]:
plot = plt.figure(figsize=(15,7))
sns.barplot(x="quality",y="pH", data=df)

# Heatmap for correlation between all features

In [ ]:
correlation = df.corr()
plt.figure(figsize=(14,14))
sns.heatmap(correlation, cbar=True, square=True, fmt = '.2f', annot = True, annot_kws={'size':15}, cmap = 'coolwarm')

# **Treating outliers**

I am going to use IQR method remove the outlier. The value which is above the Q3+ 1.5IQR and below Q1-1.5IQR are considers as Outliers.
So we are removeing all those value from selected columns.

In [ ]:
cols = ['fixed acidity', 'volatile acidity', 'citric acid','residual sugar', 'chlorides', 'free sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']

Q1 = df[cols].quantile(0.25)
Q3 = df[cols].quantile(0.75)
IQR = Q3 - Q1

df = df[~((df[cols] < (Q1 - 1.5 * IQR)) |(df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

# **Checking for skewness**

Skewd diustribution gives bias result on training. So it need to be fixed before training it.

We will first check the data for each features is following normal distribution or not using distplot.

if it is not following then we will use the boxcox to treat the skewness.
boxcox transofrm the skewed data into normal distribution.

**1. fixed acidity**

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.distplot(df["fixed acidity"], fit=norm, color="orange")
plt.title("fixed acidity Distplot", color = "darkred")
plt.subplot(1,2,2)
stats.probplot(df["fixed acidity"], plot = plt)
plt.show()

**Fixing skewness using boxcox**

In [ ]:
df["fixed acidity"], lam = boxcox(df["fixed acidity"])

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.distplot(df["fixed acidity"], fit=norm, color="orange")
plt.title("fixed acidity Distplot", color = "darkred")
plt.subplot(1,2,2)
stats.probplot(df["fixed acidity"], plot = plt)
plt.show()

**2. residual sugar**

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.distplot(df["residual sugar"], fit=norm, color="orange")
plt.title("residual sugar Distplot", color = "darkred")
plt.subplot(1,2,2)
stats.probplot(df["residual sugar"], plot = plt)
plt.show()

**Fixing skewness using boxcox**

In [ ]:
df["residual sugar"], lam_fixed_acidity = boxcox(df["residual sugar"])

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.distplot(df["residual sugar"], fit=norm, color="orange")
plt.title("residual sugar Distplot", color = "darkred")
plt.subplot(1,2,2)
stats.probplot(df["residual sugar"], plot = plt)
plt.show()

**3. free sulfur dioxide**

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.distplot(df["free sulfur dioxide"], fit=norm, color="orange")
plt.title("free sulfur dioxide Distplot", color = "darkred")
plt.subplot(1,2,2)
stats.probplot(df["free sulfur dioxide"], plot = plt)
plt.show()

In [ ]:
#Fixing skewness using boxcox
df["free sulfur dioxide"], lam_fixed_acidity = boxcox(df["free sulfur dioxide"])

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.distplot(df["free sulfur dioxide"], fit=norm, color="orange")
plt.title("free sulfur dioxide Distplot", color = "darkred")
plt.subplot(1,2,2)
stats.probplot(df["free sulfur dioxide"], plot = plt)
plt.show()

**4. total sulfur dioxide**

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.distplot(df["total sulfur dioxide"], fit=norm, color="orange")
plt.title("total sulfur dioxide Distplot", color = "darkred")
plt.subplot(1,2,2)
stats.probplot(df["total sulfur dioxide"], plot = plt)
plt.show()

In [ ]:
#Fixing skewness using boxcox
df["total sulfur dioxide"], lam_fixed_acidity = boxcox(df["total sulfur dioxide"])

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.distplot(df["total sulfur dioxide"], fit=norm, color="orange")
plt.title("total sulfur dioxide Distplot", color = "darkred")
plt.subplot(1,2,2)
stats.probplot(df["total sulfur dioxide"], plot = plt)
plt.show()

**5. alcohol**

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.distplot(df["alcohol"], fit=norm, color="orange")
plt.title("alcohol Distplot", color = "darkred")
plt.subplot(1,2,2)
stats.probplot(df["alcohol"], plot = plt)
plt.show()

In [ ]:
#Fixing skewness using boxcox
df["alcohol"], lam_fixed_acidity = boxcox(df["alcohol"])

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.distplot(df["alcohol"], fit=norm, color="orange")
plt.title("alcohol Distplot", color = "darkred")
plt.subplot(1,2,2)
stats.probplot(df["alcohol"], plot = plt)
plt.show()

# **Preprocessing the data**

In [ ]:
X = df.drop('quality',axis=1)

# Multiclassification

In [ ]:
# dividing the target variable (quality) into 3 class.
# class1: bad (qulaity 3,4), Class2: Avergae (qulaity 5,6,7), Class3: Good (qulaity 8)
reviews = []
for i in df['quality']:
    if i >= 3 and i <= 4:
        reviews.append('1')
    elif i >= 5 and i <= 7:
        reviews.append('2')
    elif i == 8:
        reviews.append('3')
df['Reviews'] = reviews

In [ ]:
Counter(df['Reviews'])

# **Standarisation**

**Why we need standarisation?**
Data standardization is about making sure that data is internally consistent; that is, each data type has the same content and format.

For example, suppose you and your friend went to different universities. One day, you both got your midterm grades for your Physics 101 classes. Your professor sticks to the normal grading scale (out of 100), so you got a grade of 84; the test has a mean of 77 and a standard deviation of 6. Your friend’s professor, though, uses his own grading scale, so she got a grade of 452. Her test has a scale of 750, mean of 400, and standard deviation of 100.

Both of you scored above average, but who did better? While the main data points might not be immediately comparable, there is a way to standardize and compare the data points. Converting them to percentages shows that you came out ahead, with an 84% compared to your friend’s 60%.

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)


# **PCA for dimension reduction**

pca removes features which has less contribiuton in 

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
X_pca = pca.fit_transform(X)


In [ ]:
#plot the graph to find the principal components
plt.figure(figsize=(10,10))
plt.plot(np.cumsum(pca.explained_variance_ratio_), 'ro-')
plt.grid()

In [ ]:
#AS per the graph, we can see that 8 principal components attribute for 90% of variation in the data. 
#we shall pick the first 8 components for our prediction.
pca_new = PCA(n_components=8)
X_new = pca_new.fit_transform(X)


**Spliting the data in train and test**

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_new, reviews, test_size = 0.25)

# **SMOTE**

**fixing the imbalance dataset**

we need to fix imbaalnce dataset becuase imbalamce dataset gives biased result

In [ ]:
sm = SMOTE(random_state=14)
X_train, Y_train = sm.fit_resample(X_train, Y_train)

In [ ]:
import collections
print("change is value of count after using smote:", collections.Counter(Y_train))

# Training the data

**Random forest classifier**

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, Y_train)

**Checking the accuracy**

In [ ]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy : ', test_data_accuracy)